In [ ]:
# Librerías
# !pip install imaplib email

In [1]:
import imaplib
import email
from email.header import decode_header
import os
from datetime import datetime, timedelta

# Configuración de la cuenta
IMAP_SERVER = "imap-mail.outlook.com"
EMAIL_ACCOUNT = "facturas_gpf@outlook.com"
APP_PASSWORD = "lleibtocysmvsnko"

# Conectar al servidor IMAP
mail = imaplib.IMAP4_SSL(IMAP_SERVER)

# Iniciar sesión
mail.login(EMAIL_ACCOUNT, APP_PASSWORD)

# Seleccionar la bandeja de entrada
mail.select("inbox")

# Obtener las fechas de hoy y ayer
date_today = datetime.now().strftime("%d-%b-%Y")
date_yesterday = (datetime.now() - timedelta(1)).strftime("%d-%b-%Y")

# Buscar correos del día de hoy y del día de ayer
status, messages_today = mail.search(None, f'(ON {date_today})')
status, messages_yesterday = mail.search(None, f'(ON {date_yesterday})')

# Combinar mensajes de hoy y ayer
messages = messages_today[0].split() + messages_yesterday[0].split()

for msg_num in messages:
    # Obtener el mensaje
    status, msg_data = mail.fetch(msg_num, "(RFC822)")
    msg = email.message_from_bytes(msg_data[0][1])

    # Obtener el asunto del mensaje
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding if encoding else "utf-8")

    # Crear una carpeta con el nombre del asunto
    folder_name = subject.replace("/", "_")  # Evitar problemas con nombres de carpeta
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    # Iterar sobre las partes del mensaje
    for part in msg.walk():
        if part.get_content_maintype() == "multipart":
            continue
        if part.get("Content-Disposition") is None:
            continue

        filename = part.get_filename()
        if filename:
            filepath = os.path.join(folder_name, filename)
            with open(filepath, "wb") as f:
                f.write(part.get_payload(decode=True))

# Cerrar la conexión
mail.close()
mail.logout()



('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])